In [1]:
# imports

import os
import re
import math
import json
from tqdm import tqdm
import random
from dotenv import load_dotenv
from huggingface_hub import login
import numpy as np
import pickle
from sentence_transformers import SentenceTransformer
from datasets import load_dataset
import chromadb
from sklearn.manifold import TSNE
import plotly.graph_objects as go

In [2]:
# environment

load_dotenv(override=True)
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')
os.environ['HF_TOKEN'] = os.getenv('HF_TOKEN', 'your-key-if-not-using-env')
DB = "products_vectorstore"

In [3]:
# Log in to HuggingFace

hf_token = os.environ['HF_TOKEN']
login(hf_token, add_to_git_credential=True)

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [4]:
# Another import after Logging in to Hugging Face - thank you Trung N.!

from items import Item

In [10]:
import pickle
from pathlib import Path

base = Path.cwd()          # this is week8 folder
week6 = base.parent / "week6"

with open(week6 / "train_lite.pkl", "rb") as file:
    train = pickle.load(file)

with open(week6 / "test_lite.pkl", "rb") as file:
    test = pickle.load(file)


In [11]:
train[0].prompt

'How much does this cost to the nearest dollar?\n\nand Replacement Range Cooktop Drip Pans fit GE, Hotpoint - Two 6 Inch and Two 8 Inch Pans (4 pieces)\nContents 2 x (6 inches) and 2 x (8 inches) bowls, 4 drip bowls total Compatibility This replacement kit works with GE, Hotpoint, Moffat, Monogram (GE), Profile (GE), RCA (GE), and Roper models prior to 1996. replaces 65975, replaces and 65974, 770169 Premium quality Drip bowls are made of durable high-quality material. It features a chrome finish, well-tested by the manufacturer. Durable, stick-free, easy to clean, and dishwasher safe. Ensure long-lasting and effective performance Easy to install Shut off electrical power, tilt the coil\n\nPrice is $12.00'

In [ ]:
client = chromadb.PersistentClient(path=DB)#

In [13]:
# Check if the collection exists and delete it if it does
collection_name = "products"

# For old versions of Chroma, use this line instead of the subsequent one
# existing_collection_names = [collection.name for collection in client.list_collections()]
existing_collection_names = client.list_collections()

if collection_name in existing_collection_names:
    client.delete_collection(collection_name)
    print(f"Deleted existing collection: {collection_name}")

collection = client.create_collection(collection_name)

In [14]:
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

In [15]:
# Pass in a list of texts, get back a numpy array of vectors

vector = model.encode(["Well hi there"])[0]
vector

array([-9.46715400e-02,  4.27619331e-02,  5.51620722e-02, -5.10978920e-04,
        1.16203260e-02, -6.80130646e-02,  2.76405718e-02,  6.06974475e-02,
        2.88530383e-02, -1.74127743e-02, -4.94346097e-02,  2.30993573e-02,
       -1.28614437e-02, -4.31402922e-02,  2.17510499e-02,  4.26548868e-02,
        5.10500148e-02, -7.79727027e-02, -1.23247236e-01,  3.67455333e-02,
        4.54114517e-03,  9.47937816e-02, -5.53098582e-02,  1.70641579e-02,
       -2.92873103e-02, -4.47125025e-02,  2.06784159e-02,  6.39320463e-02,
        2.27427855e-02,  4.87789474e-02, -2.33501708e-03,  4.72859479e-02,
       -2.86259186e-02,  2.30624508e-02,  2.45130192e-02,  3.95681709e-02,
       -4.33176309e-02, -1.02316633e-01,  2.79875379e-03,  2.39304882e-02,
        1.61556583e-02, -8.99077859e-03,  2.07255725e-02,  6.40123338e-02,
        6.89179003e-02, -6.98360950e-02,  2.89760437e-03, -8.10989067e-02,
        1.71123147e-02,  2.50655157e-03, -1.06529072e-01, -4.87733036e-02,
       -1.67762395e-02, -

In [16]:
# Quick sidebar - extra to the videos - a function to compare vectors

import numpy as np
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

def how_similar(text1, text2):
    vector1, vector2 = model.encode([text1, text2])
    similarity = cosine_similarity(vector1, vector2)
    print(f"Similarity between {text1} and {text2} is {similarity*100:.1f}%")

In [17]:
# And let's see how adding a few words to the context can change things up!

how_similar("Java", "C++")
how_similar("Java", "mug")
how_similar("Cup of Java", "mug")

Similarity between Java and C++ is 50.7%
Similarity between Java and mug is 25.8%
Similarity between Cup of Java and mug is 49.3%


In [18]:
# OK back to the main story - let's make something we can vectorize

def description(item):
    text = item.prompt.replace("How much does this cost to the nearest dollar?\n\n", "")
    return text.split("\n\nPrice is $")[0]

In [19]:
description(train[0])

'and Replacement Range Cooktop Drip Pans fit GE, Hotpoint - Two 6 Inch and Two 8 Inch Pans (4 pieces)\nContents 2 x (6 inches) and 2 x (8 inches) bowls, 4 drip bowls total Compatibility This replacement kit works with GE, Hotpoint, Moffat, Monogram (GE), Profile (GE), RCA (GE), and Roper models prior to 1996. replaces 65975, replaces and 65974, 770169 Premium quality Drip bowls are made of durable high-quality material. It features a chrome finish, well-tested by the manufacturer. Durable, stick-free, easy to clean, and dishwasher safe. Ensure long-lasting and effective performance Easy to install Shut off electrical power, tilt the coil'

In [20]:
NUMBER_OF_DOCUMENTS = len(train)

# Uncomment if you'd rather not wait for the full 400,000
# NUMBER_OF_DOCUMENTS = 20000

for i in tqdm(range(0, NUMBER_OF_DOCUMENTS, 1000)):
    documents = [description(item) for item in train[i: i+1000]]
    vectors = model.encode(documents).astype(float).tolist()
    metadatas = [{"category": item.category, "price": item.price} for item in train[i: i+1000]]
    ids = [f"doc_{j}" for j in range(i, i+len(documents))]
    collection.add(
        ids=ids,
        documents=documents,
        embeddings=vectors,
        metadatas=metadatas
    )

100%|██████████| 25/25 [03:57<00:00,  9.52s/it]
